In [1]:
from torch.utils.data import Dataset, DataLoader
from abc import abstractmethod
from nltk import sent_tokenize
import torch.nn as nn
import pandas as pd
import numpy as np
import itertools
import argparse
import easydict
import random
import pickle
import MeCab
import torch
import json
import sys
import os
import re

In [4]:
def config_parser(args):
    print('file path is ' + str(args.file_path))
    with open(args.config_path, 'rb') as f:
        config = easydict.EasyDict(json.load(f))
    config.model = args.model
    config.file_path = args.file_path
    config.dataset_path = args.dataset_path
    config.device = torch.device(args.device)
    return config

args = argparse.ArgumentParser(description='nlp embedding')
args.add_argument('-m', '--model', default='neg-sampling', type=str,
                  help='which model to use')
args.add_argument('-cp', '--config-path', default='config.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-fp', '--file-path', default='D:\\data\\text\\news-articles\\kbanker_articles_subtitles.csv', type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-dp', '--dataset-path', default='D:\\data\\text\\torch-dataset\\kbanker_articles_processed.pkl', type=str,
                  help='if there is a pickled dataset')
args.add_argument('-d', '--device', default='cuda:0', type=str,
                  help='indices of GPUs to enable (default: all)')
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
config = config_parser(args.parse_args())
config['neg_sample_size'] = 5
config['file_path'] = 'D:\\data\\text\\news-articles\\kbanker_articles_subtitles.csv'
config['file_path'] = 'D:\\data\\text\\news-articles\\kbanker_articles_processed.pkl'

file path is C:\Users\sylim2357\AppData\Roaming\jupyter\runtime\kernel-3fd747c8-c7cc-4136-a34a-0ba6b1bc4285.json


In [5]:
config

{'batch_size': 128,
 'epoch': 20,
 'gpu': 0,
 'window_size': 2,
 'embed_dim': 20,
 'h_dim': 500,
 'neg_sample_size': 5,
 'model': 'neg-sampling',
 'file_path': 'D:\\data\\text\\news-articles\\kbanker_articles_processed.pkl',
 'dataset_path': 'data\\nlp_dataset.pkl',
 'device': device(type='cuda', index=0)}

In [4]:
#pre-processing shit
def pre_process_raw_article(article):
    """Pre-processing news articles.
    
    Args
        article (str): article text
    
    """
    replacements = [
        ('[“”]', '"'),
        ('[‘’]', '\''),
        ('\([^)]*\)', ''),
        ('[^가-힣\'"A-Za-z0-9.\s\?\!]', ' '),
        ('(?=[^0-9])\.(?=[^0-9])', '. '),
        ('\s\s+', ' ')
    ]
    
    for old, new in replacements:
        article = re.sub(old, new, article)
        
    return article

def mecab_tokenize(sentence):
    t = MeCab.Tagger()
    return [re.split(',', re.sub('\t', ',', s))[0] for s in t.parse(sentence).split('\n') if (s!='') & ('EOS' not in s)]

In [6]:
def pre_process(articles):
    print('preprocessing the corpus')
    articles = [pre_process_raw_article(article) for article in articles]
    sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
    corpus = [mecab_tokenize(s) for s in list(sentences)]
    return corpus

kbanker_processed = pre_process(pd.read_csv(config.file_path, encoding='utf-8')['article'].dropna().values)

preprocessing the corpus


In [15]:
with open('kbanker_articles_processed.pkl', 'wb') as f:
    pickle.dump(kbanker_processed, f)

In [6]:
# from w2v.utils import pre_process_raw_article, mecab_tokenize
from utils import pre_process_raw_article, mecab_tokenize
from torch.utils.data import Dataset
from nltk import sent_tokenize
from abc import abstractmethod
import pandas as pd
import collections
import itertools


class W2VDataset(Dataset):
    """W2V Dataset

    Args:
        config (dict): hyperparameters

    Attributes:
        root_dir (str): root
        word_to_idx (dict): word_to_idx mapping
        idx_to_word (dict): idx_to_word mapping
        x (list): train data (5-gram)
        y (list): label

    """

    def __init__(self, config):
        if 'pkl' in config.file_path:
            with open(config.file_path, 'rb') as f:
                corpus = pickle.load(f)[:1000]
        else:
            articles = pd.read_csv(config.file_path, encoding='utf-8')['article'].dropna().values

            #pre process
            corpus = self.pre_process(articles)

        #construct word matrix
        self.word_to_idx, self.idx_to_word = self.construct_word_idx(corpus)
        corpus = [[self.word_to_idx[word] for word in sentence] for sentence in corpus]

        #make dataset
        self.x, self.y = self.construct_dataset(corpus, config)

    def pre_process(self, articles):
        print('preprocessing the corpus')
        articles = [pre_process_raw_article(article) for article in articles]
        sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
        corpus = [mecab_tokenize(s) for s in list(sentences)]
        
        return corpus

    def construct_word_idx(self, corpus):
        print('constructing word matrix')
        word_set = set(itertools.chain.from_iterable(corpus))
        word_to_idx = {word : idx for idx, word in enumerate(word_set)}
        idx_to_word = {word_to_idx[word] : word for word in word_to_idx}
        
        return word_to_idx, idx_to_word

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
        
        return self.x[idx], self.y[idx]

    @abstractmethod
    def construct_dataset(self, corpus, config):
        raise NotImplementedError

class CBOWDataset(W2VDataset):
    """CBOW Dataset"""

    def __init__(self, config):
        super().__init__(config)

    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        x, y = [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                x += [sentence[i-config.window_size:i] + sentence[i+1:i+condig.window_size+1]]
                y += [sentence[i]]
        
        return x, y

class SkipGramDataset(W2VDataset):
    """Skip-Gram Dataset"""

    def __init__(self, config):
        super().__init__(config)

    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        x, y = [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                x += [sentence[i]] * (config.window_size*2)
                y += sentence[i-config.window_size:i] + sentence[i+1:i+config.window_size+1]
        
        return x, y

class NegSamplingDataset(W2VDataset):
    """Negative Sampling Dataset.

    Args:
        config (dict): hyperparameters
        word_frequency (dict): word index - word frequency map for negative sampling
        
    """

    def __init__(self, config):
        super().__init__(config)

    def construct_word_idx(self, corpus):
        print('constructing word matrix')
        word_frequency = collections.Counter(itertools.chain.from_iterable(corpus))
        word_frequency = {word: word_frequency[word]**(3/4) for idx, word in enumerate(word_frequency)}
        word_to_idx = {word: idx for idx, word in enumerate(word_frequency)}
        idx_to_word = {word_to_idx[word]: word for word in word_to_idx}
        self.word_frequency = {word_to_idx[word]: word_frequency[word] for word in word_frequency}
        
        return word_to_idx, idx_to_word
    
    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        target, pos, neg = [], [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                target += [sentence[i]] * (config.window_size*2)
                pos += sentence[i-config.window_size:i] + sentence[i+1:i+config.window_size+1]
                neg.append(self.neg_sample(sentence[i-config.window_size:i+config.window_size+1], config))
        neg = np.array(neg).reshape(-1, config.window_size*2)
        
        return (pos, neg), target
    
    def neg_sample(self, word_contxt, config):
        word_universe = self.idx_to_word.keys() - set(word_contxt)
        word_distn = np.array([self.word_frequency[idx] for idx in word_universe])
        word_distn = word_distn / word_distn.sum()
        
        return np.random.choice(a=list(word_universe), size=config.neg_sample_size*config.window_size*2, p=word_distn)

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
            
        return [self.x[0][idx], self.y[idx], 1], [self.x[1][idx], self.y[idx], 0]

In [7]:
neg_sam_dataset = NegSamplingDataset(config)

constructing word matrix
constructing training dataset


In [5]:
import pickle
# with open('kbanker_nlp_dataset.pkl', 'wb') as f:
#     pickle.dump(nlp_dataset, f)
    
with open('D:\\data\\text\\torch-dataset\\kbanker_nlp_dataset.pkl', 'rb') as f:
    nlp_dataset = pickle.load(f)

In [8]:
def collate_fn(data):
    seqs, labels = zip(*data)
    return seqs, labels

dataloader = DataLoader(neg_sam_dataset, batch_size=config.batch_size, \
                        shuffle=False, num_workers=0)

target_emb = nn.Embedding(len(neg_sam_dataset), config.embed_dim).float().to(config.device)
context_emb = nn.Embedding(len(neg_sam_dataset), config.embed_dim).float().to(config.device)
target_emb.weight.data.uniform_(-1,1)
context_emb.weight.data.uniform_(-1,1)
sigmoid = nn.Sigmoid()
similar = nn.CosineSimilarity()

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(list(target_emb.parameters()) + list(context_emb.parameters()), lr=5e-3, momentum=0.9)

for epoch in range(500):
    print('Epoch ' + str(epoch))
    for i, sample in enumerate(dataloader):
        pos_idx = sample[0][0].long().to(config.device)
        neg_idx = sample[1][0].long().to(config.device)
        target_idx = sample[0][1].long().to(config.device)
        pos_label = sample[0][2].float().to(config.device)
        neg_label = sample[1][2].float().to(config.device)
        
        pos = context_emb(pos_idx)
        neg = context_emb(neg_idx)
        target = target_emb(target_idx)
        
        pred_pos = sigmoid(similar(pos, target))
        pred_neg = sigmoid(similar(neg, target.unsqueeze(1).expand_as(neg)))

        pos_loss = criterion(pred_pos, pos_label)
        neg_loss = torch.sum(criterion(pred_neg, neg_label.unsqueeze(1).expand_as(pred_neg)))

        pos_loss.retain_grad()
        neg_loss.retain_grad()
        
        loss = pos_loss + neg_loss
        loss.retain_grad()
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        if i % 100 == 99:
            print(i, loss.item())
            if i % 100000 == 99:
                with open('./checkpoints/neg_sample_checkpoint_epoch' + str(epoch) + '_' + str(i) + '.pkl', 'wb') as f:
                    pickle.dump(target_emb, f)
                print('./checkpoints/neg_sample_checkpoint_epoch' + str(epoch) + '_' + str(i) + '.pkl saved')

Epoch 0
99 1.4183127880096436
./checkpoints/neg_sample_checkpoint_epoch0_99.pkl saved
199 1.4141017198562622
299 1.4031469821929932
399 1.4308102130889893
499 1.4169652462005615
599 1.4233850240707397
699 1.4330205917358398
799 1.414907455444336
899 1.434909462928772
Epoch 1
99 1.4182100296020508
./checkpoints/neg_sample_checkpoint_epoch1_99.pkl saved
199 1.4139089584350586
299 1.4027708768844604
399 1.4305691719055176
499 1.4168322086334229
599 1.4230918884277344
699 1.4333142042160034
799 1.4147827625274658
899 1.4343334436416626
Epoch 2
99 1.4175679683685303
./checkpoints/neg_sample_checkpoint_epoch2_99.pkl saved
199 1.4137201309204102
299 1.4021393060684204
399 1.4305514097213745
499 1.4163631200790405
599 1.4227967262268066
699 1.4332304000854492
799 1.4146549701690674
899 1.4340304136276245
Epoch 3
99 1.4174610376358032
./checkpoints/neg_sample_checkpoint_epoch3_99.pkl saved
199 1.4135329723358154
299 1.4017672538757324
399 1.4296211004257202
499 1.4158947467803955
599 1.42250216

399 1.4239583015441895
499 1.4026703834533691
599 1.4166338443756104
699 1.4315069913864136
799 1.4092817306518555
899 1.4277873039245605
Epoch 31
99 1.415958046913147
./checkpoints/neg_sample_checkpoint_epoch31_99.pkl saved
199 1.4082374572753906
299 1.3942162990570068
399 1.4236829280853271
499 1.4023090600967407
599 1.415723443031311
699 1.431382656097412
799 1.4091284275054932
899 1.4275169372558594
Epoch 32
99 1.4158117771148682
./checkpoints/neg_sample_checkpoint_epoch32_99.pkl saved
199 1.4080870151519775
299 1.3939136266708374
399 1.4234068393707275
499 1.4017475843429565
599 1.4150855541229248
699 1.4312559366226196
799 1.4089725017547607
899 1.427248239517212
Epoch 33
99 1.4156651496887207
./checkpoints/neg_sample_checkpoint_epoch33_99.pkl saved
199 1.4085464477539062
299 1.3932244777679443
399 1.423130750656128
499 1.4012744426727295
599 1.4148316383361816
699 1.431127905845642
799 1.4088139533996582
899 1.4269808530807495
Epoch 34
99 1.415517807006836
./checkpoints/neg_samp

899 1.4207539558410645
Epoch 61
99 1.410974383354187
./checkpoints/neg_sample_checkpoint_epoch61_99.pkl saved
199 1.4081388711929321
299 1.38621985912323
399 1.4145495891571045
499 1.391650915145874
599 1.4078001976013184
699 1.426344394683838
799 1.4050943851470947
899 1.420504093170166
Epoch 62
99 1.4108054637908936
./checkpoints/neg_sample_checkpoint_epoch62_99.pkl saved
199 1.4080173969268799
299 1.3858678340911865
399 1.4149892330169678
499 1.3911889791488647
599 1.4075639247894287
699 1.4270557165145874
799 1.4049293994903564
899 1.4205331802368164
Epoch 63
99 1.410634994506836
./checkpoints/neg_sample_checkpoint_epoch63_99.pkl saved
199 1.4078953266143799
299 1.3856234550476074
399 1.4146952629089355
499 1.39072847366333
599 1.4073712825775146
699 1.4272639751434326
799 1.4046975374221802
899 1.4202830791473389
Epoch 64
99 1.4104634523391724
./checkpoints/neg_sample_checkpoint_epoch64_99.pkl saved
199 1.4077736139297485
299 1.3853809833526611
399 1.414868950843811
499 1.39026880

199 1.4047876596450806
299 1.3796958923339844
399 1.408289909362793
499 1.380185604095459
599 1.4008169174194336
699 1.4227380752563477
799 1.3981044292449951
899 1.4138991832733154
Epoch 92
99 1.4064078330993652
./checkpoints/neg_sample_checkpoint_epoch92_99.pkl saved
199 1.4046604633331299
299 1.3794832229614258
399 1.407982349395752
499 1.3797707557678223
599 1.4000694751739502
699 1.422499418258667
799 1.397806167602539
899 1.4136191606521606
Epoch 93
99 1.405465841293335
./checkpoints/neg_sample_checkpoint_epoch93_99.pkl saved
199 1.4045324325561523
299 1.3792729377746582
399 1.4076743125915527
499 1.3793586492538452
599 1.3998310565948486
699 1.4222412109375
799 1.3975133895874023
899 1.413336992263794
Epoch 94
99 1.4049426317214966
./checkpoints/neg_sample_checkpoint_epoch94_99.pkl saved
199 1.4032902717590332
299 1.3789011240005493
399 1.4066935777664185
499 1.3792839050292969
599 1.399595856666565
699 1.422156572341919
799 1.3972175121307373
899 1.4131065607070923
Epoch 95
99 

499 1.3733665943145752
599 1.3928191661834717
699 1.4128923416137695
799 1.3889095783233643
899 1.4083330631256104
Epoch 122
99 1.4023040533065796
./checkpoints/neg_sample_checkpoint_epoch122_99.pkl saved
199 1.3995044231414795
299 1.3742246627807617
399 1.398728609085083
499 1.3730350732803345
599 1.3925693035125732
699 1.4125900268554688
799 1.388582468032837
899 1.4080363512039185
Epoch 123
99 1.4021062850952148
./checkpoints/neg_sample_checkpoint_epoch123_99.pkl saved
199 1.3993556499481201
299 1.3740326166152954
399 1.3984373807907104
499 1.3727061748504639
599 1.3923187255859375
699 1.4122858047485352
799 1.388253927230835
899 1.4077386856079102
Epoch 124
99 1.401908278465271
./checkpoints/neg_sample_checkpoint_epoch124_99.pkl saved
199 1.3992059230804443
299 1.37384033203125
399 1.3979735374450684
499 1.3723793029785156
599 1.3920668363571167
699 1.4119799137115479
799 1.3879239559173584
899 1.4074398279190063
Epoch 125
99 1.401709794998169
./checkpoints/neg_sample_checkpoint_ep

...

399 1.3479821681976318
499 1.3246233463287354
599 1.3319984674453735
699 1.3503953218460083
799 1.3262004852294922
899 1.3516671657562256
Epoch 364
99 1.3506202697753906
./checkpoints/neg_sample_checkpoint_epoch364_99.pkl saved
199 1.3448939323425293
299 1.337580680847168
399 1.3477013111114502
499 1.3244411945343018
599 1.3317521810531616
699 1.350193977355957
799 1.3259291648864746
899 1.3515046834945679
Epoch 365
99 1.3503727912902832
./checkpoints/neg_sample_checkpoint_epoch365_99.pkl saved
199 1.3446235656738281
299 1.3373687267303467
399 1.3474200963974
499 1.324770212173462
599 1.3315078020095825
699 1.3496413230895996
799 1.3256586790084839
899 1.3513423204421997
Epoch 366
99 1.350126028060913
./checkpoints/neg_sample_checkpoint_epoch366_99.pkl saved
199 1.3443514108657837
299 1.3371608257293701
399 1.347139835357666
499 1.324589490890503
599 1.3312671184539795
699 1.349527359008789
799 1.3253892660140991
899 1.3511799573898315
Epoch 367
99 1.3492867946624756
./checkpoints/neg_

599 1.323404312133789
699 1.345455527305603
799 1.318591594696045
899 1.3475315570831299
Epoch 394
99 1.342848539352417
./checkpoints/neg_sample_checkpoint_epoch394_99.pkl saved
199 1.336032748222351
299 1.3341703414916992
399 1.3402445316314697
499 1.3199783563613892
599 1.3231799602508545
699 1.3444195985794067
799 1.3183239698410034
899 1.3477675914764404
Epoch 395
99 1.3425947427749634
./checkpoints/neg_sample_checkpoint_epoch395_99.pkl saved
199 1.335753321647644
299 1.333963394165039
399 1.3399609327316284
499 1.3198018074035645
599 1.3229511976242065
699 1.3442320823669434
799 1.3175089359283447
899 1.3479259014129639
Epoch 396
99 1.3423418998718262
./checkpoints/neg_sample_checkpoint_epoch396_99.pkl saved
199 1.3354706764221191
299 1.333756446838379
399 1.3396772146224976
499 1.3196290731430054
599 1.3232934474945068
699 1.3430746793746948
799 1.3173589706420898
899 1.3477760553359985
Epoch 397
99 1.3420501947402954
./checkpoints/neg_sample_checkpoint_epoch397_99.pkl saved
199 

799 1.3128994703292847
899 1.3437451124191284
Epoch 424
99 1.3357632160186768
./checkpoints/neg_sample_checkpoint_epoch424_99.pkl saved
199 1.328063726425171
299 1.3290542364120483
399 1.332909107208252
499 1.3152272701263428
599 1.316389799118042
699 1.3378469944000244
799 1.3126287460327148
899 1.3435888290405273
Epoch 425
99 1.3355188369750977
./checkpoints/neg_sample_checkpoint_epoch425_99.pkl saved
199 1.3277769088745117
299 1.3288503885269165
399 1.3326280117034912
499 1.3150479793548584
599 1.3161613941192627
699 1.337641954421997
799 1.31235933303833
899 1.3434338569641113
Epoch 426
99 1.3344563245773315
./checkpoints/neg_sample_checkpoint_epoch426_99.pkl saved
199 1.327719807624817
299 1.3286473751068115
399 1.3330905437469482
499 1.3154513835906982
599 1.316245436668396
699 1.3374369144439697
799 1.3120924234390259
899 1.3432812690734863
Epoch 427
99 1.335200309753418
./checkpoints/neg_sample_checkpoint_epoch427_99.pkl saved
199 1.3279300928115845
299 1.3281325101852417
399 1

199 1.3228538036346436
299 1.3234570026397705
399 1.324941635131836
499 1.3101723194122314
599 1.3104877471923828
699 1.3336400985717773
799 1.3046196699142456
899 1.3390567302703857
Epoch 455
99 1.3274140357971191
./checkpoints/neg_sample_checkpoint_epoch455_99.pkl saved
199 1.32258939743042
299 1.3232543468475342
399 1.3239350318908691
499 1.3101494312286377
599 1.3102796077728271
699 1.3334589004516602
799 1.3043631315231323
899 1.3389242887496948
Epoch 456
99 1.3271580934524536
./checkpoints/neg_sample_checkpoint_epoch456_99.pkl saved
199 1.3223248720169067
299 1.3230524063110352
399 1.3236665725708008
499 1.3100003004074097
599 1.3100757598876953
699 1.3332788944244385
799 1.3041075468063354
899 1.338792085647583
Epoch 457
99 1.3269047737121582
./checkpoints/neg_sample_checkpoint_epoch457_99.pkl saved
199 1.322059154510498
299 1.3228507041931152
399 1.3233985900878906
499 1.3098509311676025
599 1.3098751306533813
699 1.3331003189086914
799 1.3038527965545654
899 1.3386595249176025

399 1.3238866329193115
499 1.3228907585144043
599 1.318630576133728
699 1.3475253582000732
799 1.3064773082733154
899 1.3400349617004395
Epoch 485
99 1.3406181335449219
./checkpoints/neg_sample_checkpoint_epoch485_99.pkl saved
199 1.3228580951690674
299 1.320846676826477
399 1.3236465454101562
499 1.3227410316467285
599 1.3184397220611572
699 1.3474059104919434
799 1.306246280670166
899 1.339900255203247
Epoch 486
99 1.3404035568237305
./checkpoints/neg_sample_checkpoint_epoch486_99.pkl saved
199 1.3325964212417603
299 1.3240069150924683
399 1.3233951330184937
499 1.3227293491363525
599 1.318183422088623
699 1.347273349761963
799 1.313265323638916
899 1.339780569076538
Epoch 487
99 1.3402256965637207
./checkpoints/neg_sample_checkpoint_epoch487_99.pkl saved
199 1.3322720527648926
299 1.3234878778457642
399 1.3231970071792603
499 1.322601079940796
599 1.318047046661377
699 1.3474056720733643
799 1.3130487203598022
899 1.3396503925323486
Epoch 488
99 1.3400096893310547
./checkpoints/neg_